In [1]:
#https://github.com/software-mansion/starknet.py

In [ ]:
#!pip install starknet.py

In [3]:
import time
from starknet_py.contract import Contract
# import sqlite3
import json
from starknet_py.net.full_node_client import FullNodeClient
from starknet_py.net.account.account import Account
from starknet_py.net.models.chains import StarknetChainId
from starknet_py.net.signer.stark_curve_signer import KeyPair

In [4]:
# ABI of the Nostra contract
with open('nostra_contract_abi.json') as f:
    abi = json.load(f)

contract_abi = abi["contract_abi"]
type(contract_abi)

# Address of the Nostra contract
contract_address = "0x057146f6409deb4c9fa12866915dd952aa07c1eb2752e451d7f3b042086bdeb8"

# Creates an instance of account which is already deployed
# Account using transaction version=1 (has __validate__ function)
client = FullNodeClient(node_url="https://starknet-mainnet.infura.io/v3/9c1398098092415980ad945d193fe32d")

account = Account(
    client=client,
    address="0x4321",
    key_pair=KeyPair(private_key=654, public_key=321),
    chain=StarknetChainId.SEPOLIA,
)

events_response = await client.get_events(
    address=contract_address,
    keys=[['0x34e55c1cd55f1338241b50d352f0e91c7e4ffad0e4271d64eb347589ebdfd16']], # for mint event
    from_block_number=647121,
    to_block_number="latest",
    follow_continuation_token=True,
    chunk_size=47,
)

print(events_response)


total_amount_mint = 0
for i in range(0,len(events_response.events)):
    amount = events_response.events[i].data[0]
    total_amount_mint += amount
#     print(i)
#     print(total_amount/10**18)

print("mint",total_amount_mint/10**18)
    
events_response = await client.get_events(
    address=contract_address,
    keys=[['0x243e1de00e8a6bc1dfa3e950e6ade24c52e4a25de4dee7fb5affe918ad1e744']], # for burn event
    from_block_number=647121,
    to_block_number="latest",
    follow_continuation_token=True,
    chunk_size=47,
)


total_amount_burn = 0
for i in range(0,len(events_response.events)):
    amount = events_response.events[i].data[0]
    total_amount_burn += amount
#     print(i)
#     print(total_amount/10**18)

print('burn',total_amount_burn/10**18)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



burn 23065.83180022274


In [5]:
# Connect to the SQLite database (or create it)
conn = sqlite3.connect('mint_events.db')
c = conn.cursor()

# Create table for storing mint events
c.execute('''CREATE TABLE IF NOT EXISTS mints
             (tx_hash TEXT PRIMARY KEY, 
              block_number INTEGER, 
              minter TEXT, 
              amount INTEGER, 
              timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')
conn.commit()

# Define the event filter
event_filter = contract.events.Mint.createFilter(fromBlock='latest')

# Function to handle new events
def handle_event(event):
    tx_hash = event['transactionHash'].hex()
    block_number = event['blockNumber']
    minter = event['args']['minter']
    amount = event['args']['amount']
    
    # Insert event into the database
    c.execute("INSERT OR IGNORE INTO mints (tx_hash, block_number, minter, amount) VALUES (?, ?, ?, ?)",
              (tx_hash, block_number, minter, amount))
    conn.commit()

# Polling for new events
while True:
    for event in event_filter.get_new_entries():
        handle_event(event)

    # Sleep for a bit before polling again
    time.sleep(10)

NameError: name 'sqlite3' is not defined